# Merge MVO RESP files to master StationXML
This functionality is now included in libMVO/respfiles2masterstationxml

In [2]:
# Merge Montserrat RESP files
import os, sys
import pandas as pd
import glob
from obspy.io.xseed.core import _read_resp
LIBpath = os.path.join( os.getenv('HOME'),'src','kitchensinkGT', 'LIB')
sys.path.append(LIBpath)
from libMVO import fix_trace_id, inventory_fix_id_mvo, parse_STATION0HYP
from libseisGT import create_dummy_inventory, merge_inventories
SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO')
SEISAN_DB = 'MVOE_'

station0hypfile = os.path.join(SEISAN_DATA, 'DAT', 'STATION0_MVO.HYP')
station_locationsDF = parse_STATION0HYP(station0hypfile) 
print(station_locationsDF)
CALDIR = '.'
respfiles = glob.glob(os.path.join(CALDIR, 'RESP*'))
master_inv = create_dummy_inventory()
for respfile in respfiles:
    this_inv = None
    print('RESP file = ',respfile)
    this_inv = _read_resp(respfile)
    this_inv = inventory_fix_id_mvo(this_inv)
    sta_code = this_inv.networks[0].stations[0].code
    location = station_locationsDF[station_locationsDF['name']==sta_code].iloc[0].to_dict()
    #print(location)
    for station in this_inv.networks[0].stations:
        station.latitude = location['lat']
        station.longitude = location['lon']
        station.elevation = location['elev']
        for channel in station.channels:
            channel.latitude = location['lat']
            channel.longitude = location['lon']
            channel.elevation = location['elev']
            channel.depth = 0.0
            if channel.sample_rate==75.19:
                channel.sample_rate=75.0
    merge_inventories(master_inv, this_inv)
print(master_inv)    

master_inv.write("MontserratDigitalSeismicNetwork.xml",format="STATIONXML", validate=True)

    name        lat        lon   elev
0   MBBE  16.736944 -62.152500  102.0
1   MBBY  16.700278 -62.214722  161.0
2   MBFL  16.758611 -62.220833  243.0
3   MBFR  16.699444 -62.185556  541.0
4   MBGA  16.703056 -62.188056  478.0
5   MBGB  16.728333 -62.235000  253.0
6   MBGE  16.684444 -62.190000  183.0
7   MBGH  16.731667 -62.205000  350.0
8   MBHA  16.744167 -62.174444  291.0
9   MBLG  16.716667 -62.159722  287.0
10  MBLY  16.724444 -62.196111  355.0
11  MBMH  16.782778 -62.198611  270.0
12  MBRV  16.818889 -62.204444  231.0
13  MBRY  16.709444 -62.175000  355.0
14  MBSS  16.690278 -62.154444  395.0
15  MBWH  16.741944 -62.198889  407.0
16  MCPZ  16.711389 -62.190833  759.0
17  MGAT  16.716944 -62.192222  479.0
18  MGHZ  16.717222 -62.202222  351.0
19  MJHL  16.767500 -62.171944  149.0
20  MJHT  16.767500 -62.171944  149.0
21  MLGL  16.730556 -62.170556  287.0
22  MLGT  16.730556 -62.170556  287.0
23  MLYT  16.724444 -62.196111  355.0
24  MRYT  16.706389 -62.155278  355.0
25  MSPT  16